In [ ]:
import sympy as sym
from systemsim.symbolic import Equation
sym.init_printing()

## Planar UAV Single-agent modeling and IDA-PBC

In [ ]:
def planar_uav():
    #### Open Loop Dynamics  

    # Generalized coordinates and Latex display string
    q1, q2, q3 = q = sym.Matrix(sym.symbols(['q_1', 'q_2', 'q_3']))
    p1, p2, p3 = p = sym.Matrix(sym.symbols(['p_1', 'p_2', 'p_3']))

    # Symbolic parameters and display string
    symparameters = g,  epsilon,   k1,    k2,    k3,    Kv1,    Kv2  ,  Kd1   , Kd2,    base = sym.symbols(
                  ['g','epsilon', 'k_1', 'k_2', 'k_3', 'K_v1', 'K_v2', 'K_d1', 'K_d2', 'base' ])

    # System matrices
    F = sym.Matrix([[1, 0],
                    [0, 1],
                    [sym.cos(q3)/epsilon, sym.sin(q3)/epsilon]])
    M = sym.eye(3)
    Minv = M.inv()
    V = sym.Matrix([g/epsilon*sym.cos(q3)])
    H = p.T*Minv*p/2+V
    dH_dq = sym.Matrix([H]).jacobian(q).T
    dV_dq = sym.Matrix([V]).jacobian(q).T       
    n, m = F.shape

    #### IDA-PBC

    # Input matrix annihilator (Acosta et al. 2005)
    Fp = sym.Matrix([[sym.cos(q3), sym.sin(q3), -epsilon]])

    # Desired loop mass matrix (Acosta et al. 2005)
    Md =sym.Matrix([
        [k1*epsilon*sym.cos(q3)**2 + k3,
        k1*epsilon*sym.cos(q3)*sym.sin(q3),
        k1*sym.cos(q3)],
        [k1*epsilon*sym.cos(q3)*sym.sin(q3),
        -k1*epsilon*sym.cos(q3)**2 + k3,
        k1*sym.sin(q3)],
        [k1*sym.cos(q3),
        k1*sym.sin(q3),
        k2]
    ])

    Mdinv = sym.ImmutableMatrix(sym.simplify(Md.inv()))

    # J matrix (Acosta et al. 2005)
    gamma30 = k1-epsilon*k2
    J1 = (p.T*Mdinv*sym.Matrix([
        [-2*epsilon*sym.cos(q3)],
        [2*epsilon*sym.sin(q3)],
        [1]]))[0]
    J2 = (p.T*Mdinv*sym.Matrix([[ 0],[1],[0]]))[0]
    J3 = (p.T*Mdinv*sym.Matrix([[-1],[0],[0]]))[0]
    J = -k1*gamma30/2*sym.Matrix([
        [0,   J1 , J2],
        [-J1, 0  , J3],
        [-J2, -J3, 0 ]
    ])

    # Stabilization potential (Acosta et al. 2005)
    Vs = sym.Matrix([g/gamma30*(1-sym.cos(q3))])

    # Coordinate transformations (Acosta et al. 2005)
    a1 = q1 - k3/gamma30*sym.sin(q3)
    a2 = q2 - (k3-k1*epsilon)/gamma30*(1-sym.cos(q3))

    # Tracking and interaction
    z = sym.Matrix([[a1],[a2]])
    Psi = sym.Matrix([z]).jacobian(q).T

    # Gradients
    Hd = p.T*Mdinv*p/2+Vs
    dHd_dq = Hd.jacobian(q).T
    dVs_dq = Vs.jacobian(q).T
    dpMinvp_dq = (dH_dq-dV_dq)*2
    dVs_dq = Vs.jacobian(q).T
    dpMdinvp_dq = (dHd_dq-dVs_dq)*2

    #### Make numeric model equations

    # Kinematics map for animation given the coordinate q.
    # Center, right, left ends of triangle base, and apex
    center = sym.Matrix([q1, q2])
    right = center + sym.Matrix([sym.cos(q3), sym.sin(q3)])*base/2
    left = 2*center - right
    apex = center + sym.Matrix([-sym.sin(q3), sym.cos(q3)])*base/4

    # This draws a rotated triangle representing a UAV.
    kinematics = sym.Matrix([left.T, apex.T, right.T, left.T]).T

    # Generate numeric functions for simulation
    model = {
        'M': Equation(M, [q], symparameters),
        'F': Equation(F, [q], symparameters),
        'dH_dq': Equation(dH_dq, [q, p], symparameters),
        'dV_dq': Equation(dV_dq, [q], symparameters),    
        'V': Equation(V, [q], symparameters),
        'R': Equation(sym.zeros(3), [q, p], symparameters),
        'Mcl': Equation(Md, [q], symparameters),
        'Vcl': Equation(Vs, [q], symparameters),
        'dHcl_dq': Equation(dHd_dq, [q, p], symparameters),
        'dVcl_dq': Equation(dVs_dq, [q], symparameters),    
        'J2': Equation(J, [q, p], symparameters),
        'Kv': Equation(sym.diag(Kv1,Kv2), [], symparameters),
        'Kd': Equation(sym.diag(Kd1,Kd2), [], symparameters),
        'Fp': Equation(Fp, [q], symparameters, shape_as='ndarray'),
        'z': Equation(z, [q], symparameters),
        'Psi': Equation(Psi, [q], symparameters),
        'animation_kinematics': Equation(kinematics, [q], symparameters),
    }
    
    return model

## Point Mass Modeling and IDA-PBC

In [ ]:
def car():
    # Generalized coordinates and Latex display string
    q = sym.Matrix(sym.symbols(['q']))
    p = sym.Matrix(sym.symbols(['p']))


    # Symbolic parameters and display string
    symparameters = Kv,   size,   m,   md = sym.symbols(
                  ['Kv', 'size', 'm', 'md' ])

    # System matrices
    one = sym.Matrix([[1]])
    zero = sym.Matrix([[0]])
    F = one
    V = zero
    M = one*m
    H = p.T*M*p/2+V
    dH_dq = sym.Matrix([H]).jacobian(q).T
    dV_dq = sym.Matrix([V]).jacobian(q).T   

    # IDAPBC
    Md = one*md
    Mdinv = Md.inv()
    J = zero
    Vs = zero

    # Cooperative variable
    z = sym.Matrix([q,[0]])
    Psi = z.jacobian(q).T

    # Gradients
    Hd = p.T*Mdinv*p/2+Vs
    dHd_dq = Hd.jacobian(q).T
    dVs_dq = Vs.jacobian(q).T
    dpMinvp_dq = (dH_dq-dV_dq)*2
    dVs_dq = Vs.jacobian(q).T
    dpMdinvp_dq = (dHd_dq-dVs_dq)*2

    # Kinematics map for animation given the coordinate q.
    center = sym.Matrix([q, [0]])
    right = center + sym.Matrix([[size/2],[0]])
    left = center - sym.Matrix([[size/2],[0]])

    # This draws a line representing a landing vehicle
    kinematics = sym.Matrix([left.T, center.T, right.T]).T

    # Generate numeric functions for simulation
    model = {
        'M': Equation(M, [q], symparameters, shape_as='ndarray'),
        'F': Equation(F, [q], symparameters, shape_as='ndarray'),
        'dH_dq': Equation(dH_dq, [q, p], symparameters),
        'dV_dq': Equation(dV_dq, [q], symparameters),    
        'V': Equation(V, [q], symparameters),
        'R': Equation(sym.zeros(1), [q, p], symparameters, shape_as='ndarray'),
        'Mcl': Equation(Md, [q], symparameters, shape_as='ndarray'),
        'Vcl': Equation(Vs, [q], symparameters),
        'dHcl_dq': Equation(dHd_dq, [q, p], symparameters),
        'dVcl_dq': Equation(dVs_dq, [q], symparameters),    
        'J2': Equation(J, [q, p], symparameters, shape_as='ndarray'),
        'Kv': Equation(sym.diag(Kv), [], symparameters, shape_as='ndarray'),
        'Kd': Equation(sym.diag(0,0), [], symparameters, shape_as='ndarray'),
        'z': Equation(z, [q], symparameters),
        'Psi': Equation(Psi, [q], symparameters, shape_as='ndarray'),
        'animation_kinematics': Equation(kinematics, [q], symparameters),
    }
    return model

## Flexible-Joint Robot Single-agent modeling and IDA-PBC

In [ ]:
def flexible_joint_robot(links=2):
    m = links
    n = m*2
    
    # Generalized coordinates and Latex display string
    a = sym.Matrix(sym.symbols(['alpha_'+str(i) for i in range(m)]))
    b = sym.Matrix(sym.symbols(['beta_'+str(i) for i in range(m)]))
    q = a.col_join(b)
    pa = sym.Matrix(sym.symbols(['pa_'+str(i) for i in range(m)]))
    pb = sym.Matrix(sym.symbols(['pb_'+str(i) for i in range(m)]))
    p = pa.col_join(pb)

    # Symbolic parameters and display string
    symparameters = L,  mass,   I_a,   I_b,   k,   kd,   kv,   bx,    by = sym.symbols(
                  ['L','M',    'I_a', 'I_b', 'k', 'kd', 'kv', 'b_x', 'b_y'])

    # Angle and center of mass for the first link:
    gamma = a[0]
    x = L/2*sym.cos(gamma) + bx
    y = L/2*sym.sin(gamma) + by
    links = [[gamma, x, y]]
    ends = [[(x-bx)*2+bx, (y-by)*2+by]]

    # Angle and center of mass for the remaining links:
    for i in range(1,m):
        #Unpack angle and center of mass of previous link
        gamma_prev, x_prev, y_prev = links[i-1]

        # Compute angle and center of mass of current link
        gamma = gamma_prev + a[i]
        x = x_prev+L/2*sym.cos(gamma_prev)+L/2*sym.cos(gamma)
        y = y_prev+L/2*sym.sin(gamma_prev)+L/2*sym.sin(gamma)

        # Append link information
        links.append([gamma, x, y])
        ends.append([x+L/2*sym.cos(gamma), y+L/2*sym.sin(gamma)])

    # Unpack all the link angles and center of masses into one vector
    w = sym.Matrix([item for link in links for item in link])
    dw_da = w.jacobian(a)

    # end effector in alpha and beta
    z_a = sym.Matrix([x + L/2*sym.cos(gamma), y + L/2*sym.sin(gamma)])
    z_b = z = z_a.subs({a[i]:b[i] for i in range(m)})
    ell = len(z_b)

    # Static mass matrix in euclidean space
    M_static = sym.diag(*([I_a, mass, mass]*m))
    M_a = sym.trigsimp(dw_da.T*M_static*dw_da)

    M_b = I_b*sym.eye(m)
    M = sym.diag(M_a, M_b)

    # System matrices
    F = sym.zeros(m).col_join(sym.eye(m))
    stretch = b-a
    V = stretch.T*stretch*k/2

    # Avoid explicit inversion
    Ma_inv_pa = M_a.LUsolve(pa)
    Mb_inv_pb = M_b.LUsolve(pb)
    M_inv_p = Ma_inv_pa.col_join(Mb_inv_pb)
    H = p.T*M_inv_p/2+V
    dH_dq = H.jacobian(q).T
    dV_dq = V.jacobian(q).T  

    # Desired loop mass matrix
    Md = M
    Md_inv_p = M_inv_p
    J = sym.zeros(n)

    # Stabilization potential
    Vs = V

    # Tracking and interaction
    Psi = z.jacobian(q).T

    # Gradients
    Hd = p.T*Md_inv_p/2+Vs
    dHd_dq = Hd.jacobian(q).T
    dVs_dq = Vs.jacobian(q).T

    # Animation kinematics
    kinematics = sym.Matrix([bx, by])
    for i in range(m):
        kinematics = kinematics.row_join(sym.Matrix(ends[i]))

    # Generate numeric functions for simulation
    model = {
        'M': Equation(M, [q], symparameters),
        'F': Equation(F, [q], symparameters),
        'dH_dq': Equation(dH_dq, [q, p], symparameters),
        'dV_dq': Equation(dV_dq, [q], symparameters),    
        'V': Equation(V, [q], symparameters),
        'R': Equation(sym.zeros(n), [q, p], symparameters),
        'Mcl': Equation(Md, [q], symparameters),
        'Vcl': Equation(Vs, [q], symparameters),
        'dHcl_dq': Equation(dHd_dq, [q, p], symparameters),
        'dVcl_dq': Equation(dVs_dq, [q], symparameters),    
        'J2': Equation(J, [q, p], symparameters),
        'Kv': Equation(sym.eye(m)*kv, [], symparameters),
        'Kd': Equation(sym.eye(ell)*kd, [], symparameters),
        'z': Equation(z, [q], symparameters),
        'Psi': Equation(Psi, [q], symparameters),
        'animation_kinematics': Equation(kinematics, [q], symparameters),
    }

    return model

## Fully-actuated manipulator (Haptic Input Device with Scaling)

In [ ]:
def haptic_input(links=2):
    m = links
    n = m
    
    # Generalized coordinates and Latex display string
    q = sym.Matrix(sym.symbols(['q_'+str(i) for i in range(m)]))
    p = sym.Matrix(sym.symbols(['p_'+str(i) for i in range(m)]))

    # Symbolic parameters and display string
    symparameters = L,  mass,   I_a,  kd,   kv,   bx,    by  ,  scale = sym.symbols(
                  ['L','M',    'I_a','kd', 'kv', 'b_x', 'b_y', 'scale'])

    # Angle and center of mass for the first link:
    gamma = q[0]
    x = L/2*sym.cos(gamma) + bx
    y = L/2*sym.sin(gamma) + by
    links = [[gamma, x, y]]
    ends = [[(x-bx)*2+bx, (y-by)*2+by]]

    # Angle and center of mass for the remaining links:
    for i in range(1,m):
        #Unpack angle and center of mass of previous link
        gamma_prev, x_prev, y_prev = links[i-1]

        # Compute angle and center of mass of current link
        gamma = gamma_prev +q[i]
        x = x_prev+L/2*sym.cos(gamma_prev)+L/2*sym.cos(gamma)
        y = y_prev+L/2*sym.sin(gamma_prev)+L/2*sym.sin(gamma)

        # Append link information
        links.append([gamma, x, y])
        ends.append([x+L/2*sym.cos(gamma), y+L/2*sym.sin(gamma)])

    # Unpack all the link angles and center of masses into one vector
    w = sym.Matrix([item for link in links for item in link])
    dw_dq = w.jacobian(q)

    # end effector in alpha and beta
    z = (sym.Matrix([x + L/2*sym.cos(gamma), y + L/2*sym.sin(gamma)])-sym.Matrix([bx, by]))*scale
    ell = len(z)

    # Static mass matrix in euclidean space
    M_static = sym.diag(*([I_a, mass, mass]*m))
    M = sym.trigsimp(dw_dq.T*M_static*dw_dq)

    # System matrices
    F = sym.eye(m)
    V = sym.Matrix([0])

    # Avoid explicit inversion
    M_inv_p = M.LUsolve(p)
    H = p.T*M_inv_p/2+V
    dH_dq = H.jacobian(q).T
    dV_dq = V.jacobian(q).T  

    # Desired loop mass matrix
    Md = M
    Md_inv_p = M_inv_p
    J = sym.zeros(n)

    # Stabilization potential
    Vs = sym.Matrix([0])

    # Tracking and interaction
    Psi = z.jacobian(q).T

    # Gradients
    Hd = p.T*Md_inv_p/2+Vs
    dHd_dq = Hd.jacobian(q).T
    dVs_dq = Vs.jacobian(q).T

    # Animation kinematics

    kinematics = sym.Matrix([bx, by])
    for i in range(m):
        kinematics = kinematics.row_join(sym.Matrix(ends[i]))

    # Generate numeric functions for simulation
    model = {
        'M': Equation(M, [q], symparameters),
        'F': Equation(F, [q], symparameters),
        'dH_dq': Equation(dH_dq, [q, p], symparameters),
        'dV_dq': Equation(dV_dq, [q], symparameters),    
        'V': Equation(V, [q], symparameters),
        'R': Equation(sym.zeros(n), [q, p], symparameters),
        'Mcl': Equation(Md, [q], symparameters),
        'Vcl': Equation(Vs, [q], symparameters),
        'dHcl_dq': Equation(dHd_dq, [q, p], symparameters),
        'dVcl_dq': Equation(dVs_dq, [q], symparameters),    
        'J2': Equation(J, [q, p], symparameters),
        'Kv': Equation(sym.eye(m)*kv, [], symparameters),
        'Kd': Equation(sym.eye(ell)*kd, [], symparameters),
        'z': Equation(z, [q], symparameters),
        'Psi': Equation(Psi, [q], symparameters),
        'animation_kinematics': Equation(kinematics, [q], symparameters),
    }

    return model